In [1]:
import pandas as pd
import numpy as np
import os
import re

from utils import (
    write_list_datasets, PATH_UCR, ILL_FORMATED, ILL_FORMATED_DIR,
    get_fname, process_labels, get_data_labels_UCR
)

In [2]:
DATA_SOURCE = "UCR"
RES_DIR = f'../res/{DATA_SOURCE}/'

In [3]:
# Find all TRAIN files
# This gives the folder. Each folder d has the following structure
# - {d}/{d}_TRAIN.tsv
# - {d}/{d}_TEST.tsv
# - {d}/README.md
subdirs = [
    f.name for f in os.scandir(PATH_UCR)
    if f.is_dir() and f.name != ILL_FORMATED_DIR[:-1]
]
# Keep only files that have no missing values
# Save list of file names

In [4]:
all_datasets = []
for d in subdirs:
    all_datasets += [
        f"{d}/{fname}"
        for fname in os.listdir(PATH_UCR+d)
        if "TRAIN" in fname
    ]
print(len(all_datasets))
all_datasets


128


['PickupGestureWiimoteZ/PickupGestureWiimoteZ_TRAIN.tsv',
 'DistalPhalanxOutlineCorrect/DistalPhalanxOutlineCorrect_TRAIN.tsv',
 'PigArtPressure/PigArtPressure_TRAIN.tsv',
 'BirdChicken/BirdChicken_TRAIN.tsv',
 'PLAID/PLAID_TRAIN.tsv',
 'MixedShapesRegularTrain/MixedShapesRegularTrain_TRAIN.tsv',
 'SemgHandMovementCh2/SemgHandMovementCh2_TRAIN.tsv',
 'InsectEPGSmallTrain/InsectEPGSmallTrain_TRAIN.tsv',
 'InsectEPGRegularTrain/InsectEPGRegularTrain_TRAIN.tsv',
 'EOGVerticalSignal/EOGVerticalSignal_TRAIN.tsv',
 'Coffee/Coffee_TRAIN.tsv',
 'DistalPhalanxTW/DistalPhalanxTW_TRAIN.tsv',
 'HandOutlines/HandOutlines_TRAIN.tsv',
 'Yoga/Yoga_TRAIN.tsv',
 'Car/Car_TRAIN.tsv',
 'OliveOil/OliveOil_TRAIN.tsv',
 'RefrigerationDevices/RefrigerationDevices_TRAIN.tsv',
 'GesturePebbleZ2/GesturePebbleZ2_TRAIN.tsv',
 'SonyAIBORobotSurface2/SonyAIBORobotSurface2_TRAIN.tsv',
 'BME/BME_TRAIN.tsv',
 'Chinatown/Chinatown_TRAIN.tsv',
 'FiftyWords/FiftyWords_TRAIN.tsv',
 'CricketY/CricketY_TRAIN.tsv',
 'ACSF1/AC

In [5]:
# list_fname = f'{RES_DIR}all_datasets-{DATA_SOURCE}.txt'
# write_list_datasets(list_fname, subdirs)

In [7]:
def find_lines_with_patterns(filename, p1, p2):
    # Initialize a list to store the lines containing the patterns
    matching_lines = []
    lines_no = []
    lines_yes = []

    # Open the file and read its contents line by line
    with open(filename, 'r') as file:
        for line in file:
            # Use regex to search for the pattern in the line
            if re.search(p1, line):
                matching_lines.append(line)
                # If the pattern is found in the line, check for "p2"
                if p2 in line:
                    lines_no.append(line)
                else:
                    lines_yes.append(line)

    print("matching_lines", matching_lines)
    print("lines_no", lines_no)
    print("lines_yes", lines_yes)

    return matching_lines, lines_no, lines_yes

for d in subdirs:
    fname = f"{PATH_UCR}{get_fname(d, data_source=DATA_SOURCE)}"

    data, labels, n_labels, _ = get_data_labels_UCR(fname)

    print(d, n_labels, data.shape)





PickupGestureWiimoteZ 10 (49, 1, 361)
DistalPhalanxOutlineCorrect 2 (599, 1, 80)
PigArtPressure 52 (103, 1, 2000)
BirdChicken 2 (19, 1, 512)
PLAID 11 (536, 1, 1344)
MixedShapesRegularTrain 5 (499, 1, 1024)
SemgHandMovementCh2 6 (449, 1, 1500)
InsectEPGSmallTrain 3 (16, 1, 601)
InsectEPGRegularTrain 3 (61, 1, 601)
EOGVerticalSignal 12 (361, 1, 1250)
Coffee 2 (27, 1, 286)
DistalPhalanxTW 6 (399, 1, 80)
HandOutlines 2 (999, 1, 2709)
Yoga 2 (299, 1, 426)
Car 4 (59, 1, 577)
OliveOil 4 (29, 1, 570)
RefrigerationDevices 3 (374, 1, 720)
GesturePebbleZ2 6 (145, 1, 455)
SonyAIBORobotSurface2 2 (26, 1, 65)
BME 3 (29, 1, 128)
Chinatown 2 (19, 1, 24)
FiftyWords 50 (449, 1, 270)
CricketY 12 (389, 1, 300)
ACSF1 10 (99, 1, 1460)
CBF 3 (29, 1, 128)
DodgerLoopDay 7 (77, 1, 288)
InlineSkate 7 (99, 1, 1882)
FreezerSmallTrain 2 (27, 1, 301)
Adiac 37 (389, 1, 176)
PigCVP 52 (103, 1, 2000)
MiddlePhalanxTW 6 (398, 1, 80)
Earthquakes 2 (321, 1, 512)
Haptics 5 (154, 1, 1092)
GesturePebbleZ1 6 (131, 1, 455)
Dodg